<a href="https://colab.research.google.com/github/hamidpeywasti/inception-twopath/blob/main/Plantvillage38_2path5_lab_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
monitor='val_accuracy'
epochs=150
batch_size=32
input_shape=(224, 224, 3) # please resize it to (224,224,3) if you have enough RAM
Verbose=True

In [2]:
import sys
print("Python version:", sys.version)

import skimage
print('skimage:',  skimage.__version__)

import tensorflow as tf
print('Tensorflow:',tf.__version__)

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

import multiprocessing

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight

import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
skimage: 0.19.3
Tensorflow: 2.15.0


In [3]:
#apt-get install git python3-opencv
import os

if not os.path.isdir('k'):
  !git clone https://github.com/hamidpeywasti/keras-neural-api k
else:
  !cd k && git pull

!cd k && pip install .

!rm -rf k

import cai
import cai.datasets
import cai.models
import cai.inception_v3
import cai.layers
import cai.util
from cai.layers import conv2d_bn

Cloning into 'k'...
remote: Enumerating objects: 1807, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 1807 (delta 135), reused 147 (delta 72), pack-reused 1588
Receiving objects: 100% (1807/1807), 15.69 MiB | 22.85 MiB/s, done.
Resolving deltas: 100% (1248/1248), done.
Processing /content/k
  Preparing metadata (setup.py) ... done
  Created wheel for cai: filename=cai-0.1.7-py3-none-any.whl size=61388 sha256=32fbdf179a43517b4f23b40ead7589842a9bf85e52120b42a5848e4e67dccf4d
  Stored in directory: /tmp/pip-ephem-wheel-cache-58j9hzgq/wheels/80/61/f5/947bedc7e497038def7d1381fb65d37bd126a80e010114b8f1
Successfully built cai


In [4]:
def two_path5_inception_v3(
    include_top=True,
    weights=None, #'two_paths_plant_leafs'
    input_shape=(224,224,3),
    pooling=None,
    classes=1000,
    two_paths=False,
    deep_two_paths=False,
    deep_two_paths_compression=0.655,
    deep_two_paths_bottleneck_compression=0.5,
    l_ratio=0.5,
    ab_ratio=0.5,
    max_mix_idx=10,
    max_mix_deep_two_paths_idx=-1,
    model_name='two_path_inception_v3',
    kType=0,
    **kwargs
):
    """
    Instantiates the Inception v3 architecture with 2 paths options.
    """
    img_input = keras.layers.Input(shape=input_shape)
    if (deep_two_paths):  max_mix_deep_two_paths_idx = max_mix_idx

    if keras.backend.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3


    if two_paths:
        if (l_ratio>0):
            l_branch = cai.layers.CopyChannels(0,1)(img_input)
            l_branch = conv2d_bn(l_branch, int(round(32*l_ratio)), 3, 3, strides=(2, 2), padding='valid')
            l_branch = conv2d_bn(l_branch, int(round(32*l_ratio)), 3, 3, padding='valid')
            l_branch = conv2d_bn(l_branch, int(round(64*l_ratio)), 3, 3)
            l_branch = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(l_branch)

            #l_branch    = conv2d_bn(x, int(round(80*deep_two_paths_bottleneck_compression)), 1, 1, padding='valid', name='second_block_ta', activation=None, has_batch_norm=True)
            l_branch    = cai.inception_v3.create_inception_path(last_tensor=l_branch, compression=deep_two_paths_bottleneck_compression, channel_axis=channel_axis, name='second_block_ta', activation=None, has_batch_norm=True, kType=kType)
            # l_branch    = conv2d_bn(l_branch,    int(round(80 *deep_two_paths_compression)), 1, 1, padding='valid')
            l_branch = cai.inception_v3.kInceptionPointwise(l_branch, filters=int(round(80 *l_ratio)), name='l_branch_path', kType=kType)
            l_branch    = conv2d_bn(l_branch,    int(round(192*l_ratio)), 3, 3, padding='valid')

        if (ab_ratio>0):
            ab_branch = cai.layers.CopyChannels(1,2)(img_input)
            ab_branch = conv2d_bn(ab_branch, int(round(32*ab_ratio)), 3, 3, strides=(2, 2), padding='valid')
            ab_branch = conv2d_bn(ab_branch, int(round(32*ab_ratio)), 3, 3, padding='valid')
            ab_branch = conv2d_bn(ab_branch, int(round(64*ab_ratio)), 3, 3)
            ab_branch = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(ab_branch)

            #ab_branch = conv2d_bn(x, int(round(80*deep_two_paths_bottleneck_compression)), 1, 1, padding='valid', name='second_block_tb', activation=None, has_batch_norm=True)
            ab_branch = cai.inception_v3.create_inception_path(last_tensor=ab_branch, compression=deep_two_paths_bottleneck_compression, channel_axis=channel_axis, name='second_block_tb', activation=None, has_batch_norm=True, kType=kType)
            # ab_branch = conv2d_bn(ab_branch, int(round(80 *deep_two_paths_compression)), 1, 1, padding='valid')
            ab_branch = cai.inception_v3.kInceptionPointwise(ab_branch, filters=int(round(80 *ab_ratio)), name='ab_branch_path', kType=kType)
            ab_branch = conv2d_bn(ab_branch, int(round(192*ab_ratio)), 3, 3, padding='valid')

        if (l_ratio>0):
            if (ab_ratio>0):
                x = keras.layers.Concatenate(axis=channel_axis, name='concat_first_block')([l_branch, ab_branch])
            else:
                x = l_branch
        else:
            x = ab_branch

    else:
        single_branch = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
        single_branch = conv2d_bn(single_branch, 32, 3, 3, padding='valid')
        single_branch = conv2d_bn(single_branch, 64, 3, 3)
        single_branch = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(single_branch)

        # x = conv2d_bn(x, 80, 1, 1, padding='valid')
        single_branch= cai.inception_v3.kInceptionPointwise(single_branch, filters=80, name='single_path', kType=kType)
        x = conv2d_bn(single_branch, 192, 3, 3, padding='valid')


    x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    if max_mix_idx >= 0:
        for id_layer in range(max_mix_idx+1):
            if (max_mix_deep_two_paths_idx >= id_layer):
                x = cai.inception_v3.create_inception_v3_two_path_mixed_layer(x,  id=id_layer,  name='mixed'+str(id_layer),
                    channel_axis=channel_axis, bottleneck_compression=deep_two_paths_bottleneck_compression,
                    compression=deep_two_paths_compression, has_batch_norm=True, kType=kType)
            else:
                x = cai.inception_v3.create_inception_v3_mixed_layer(x,  id=id_layer,  name='mixed'+str(id_layer), channel_axis=channel_axis, kType=kType)

    if include_top:
        # Classification block
        x = keras.layers.GlobalAveragePooling2D(name='avg_pool')(x)
        x = keras.layers.Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = keras.layers.GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = keras.layers.GlobalMaxPooling2D()(x)

    inputs = img_input
    # Create model.
    model = keras.models.Model(inputs, x, name=model_name)
    return model

def plot_history(filename):
    plt.figure(figsize=(10, 6))

    # Plot training & validation loss values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.ylim(ymin=0, ymax=1.5)

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='lower right')

    plt.savefig(filename)

    plt.tight_layout()
    plt.show()

# Loading Data

In [5]:
url_zip_file="https://data.mendeley.com/datasets/tywbtsjrjv/1/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/Plant_leaf_diseases_dataset_without_augmentation.zip?dl=1"
local_zip_file="plant_leaf.zip"
expected_folder_name="plant_leaf"
Verbose=True
cai.datasets.download_zip_and_extract(
    url_zip_file=url_zip_file, local_zip_file=local_zip_file,
    expected_folder_name=expected_folder_name, Verbose=Verbose)

Downloading:  https://data.mendeley.com/datasets/tywbtsjrjv/1/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/Plant_leaf_diseases_dataset_without_augmentation.zip?dl=1  to  plant_leaf.zip
Decompressing into:  plant_leaf


In [6]:
!rm -r plant_leaf/Plant_leave_diseases_dataset_without_augmentation/Background_without_leaves -R

In [ ]:
data_dir = "plant_leaf/Plant_leave_diseases_dataset_without_augmentation/"

label_of_classes = os.listdir(data_dir)
print("Classes Lables:", label_of_classes)

number_of_classes = len(label_of_classes)
print("Number of Classes:", number_of_classes)

train_x, val_x, test_x, train_y, val_y, test_y, classweight, classes = cai.datasets.load_images_from_folders(
    seed=7,
    root_dir=data_dir,
    lab=True,
    verbose=Verbose,
    bipolar=False,
    base_model_name='plant_leaf',
    training_size=0.6,
    validation_size=0.2,
    test_size=0.2,
    target_size=(input_shape[0],input_shape[1]),
    has_training=True,
    has_validation=True,
    has_testing=True,
    smart_resize=True
)

print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

Classes Lables: ['Potato___healthy', 'Apple___Black_rot', 'Strawberry___healthy', 'Strawberry___Leaf_scorch', 'Corn___Common_rust', 'Corn___healthy', 'Corn___Northern_Leaf_Blight', 'Tomato___Septoria_leaf_spot', 'Tomato___Bacterial_spot', 'Potato___Late_blight', 'Cherry___Powdery_mildew', 'Grape___Esca_(Black_Measles)', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Tomato___Leaf_Mold', 'Pepper,_bell___healthy', 'Grape___Black_rot', 'Tomato___Late_blight', 'Orange___Haunglongbing_(Citrus_greening)', 'Squash___Powdery_mildew', 'Tomato___Target_Spot', 'Pepper,_bell___Bacterial_spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Early_blight', 'Potato___Early_blight', 'Grape___healthy', 'Peach___Bacterial_spot', 'Cherry___healthy', 'Apple___healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Blueberry___healthy', 'Peach___healthy', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Apple___Cedar_apple_rust', 'Apple___Apple_sca

# Fitting Model for L_ratio and AB_ratio

In [ ]:
for l_ratio in [0.25]:

        basefilename = 'Apple13-2Path5-LAB-m4-' + str(l_ratio)
        print('Running: '+basefilename)

        model = two_path5_inception_v3(
            include_top=True,
            weights=None,
            input_tensor=None,
            input_shape=input_shape,
            two_paths=True,
            pooling='max',
            classes=number_of_classes,
            l_ratio=l_ratio,
            ab_ratio=(1.0-l_ratio),
            max_mix_idx=4
        )

        model.compile(
            loss='categorical_crossentropy',
            optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True),
            metrics=['accuracy']
        )

        best_result_file_name = basefilename+'-best_result.hdf5'

        save_best = tf.keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=True,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch'
        )

        history = model.fit(
            train_x,
            train_y,
            epochs=120,
            batch_size=batch_size,
            validation_data=(val_x,val_y),
            callbacks=[save_best],
            class_weight=classweight,
            workers=multiprocessing.cpu_count()
        )

        history_filename = basefilename +'-History.png'
        plot_history(history_filename)

        print('Testing Last Model: '+basefilename)

        evaluated = model.evaluate(test_x,test_y)

        for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
            print(name,metric)

        # print('Best Model Results: '+basefilename)

        # model = tf.keras.models.load_model(
        #     best_result_file_name,
        #     custom_objects={'CopyChannels': cai.layers.CopyChannels}
        # )

        evaluated = model.evaluate(test_x,test_y)

        # cai.models.save_model(model, basefilename)

        # for metric, name in zip(evaluated,["loss","acc","top 5 acc"]):
        #     print(name,metric)

        print('Finished: '+basefilename)

# Calcutale F1 and other metrics

In [ ]:
for l_ratio in [0.25]:

    basefilename = 'Apple13-2Path5-LAB-m4-' + str(l_ratio)

    best_result_file_name = basefilename+'-best_result.hdf5'

    print('Best Model Results: '+basefilename)

    model = tf.keras.models.load_model(
        best_result_file_name,
        custom_objects={'CopyChannels': cai.layers.CopyChannels}
    )

    pred_y = model.predict(test_x)
    #         print("Predicted Shape:", pred_y.shape)
    pred_classes_y = np.array(list(np.argmax(pred_y, axis=1)))
    test_classes_y = np.array(list(np.argmax(test_y, axis=1)))
    #         print("Pred classes shape:",pred_classes_y.shape)
    #         print("Test classes shape:",test_classes_y.shape)
    report = classification_report(test_classes_y, pred_classes_y, digits=4)
    print(report)

# Confusion Matrix

In [ ]:
for l_ratio in [0.25]:

    basefilename = 'Apple13-2Path5-LAB-m4-' + str(l_ratio)

    best_result_file_name = basefilename+'-best_result.hdf5'

    print('Best Model Results: '+basefilename)
    model = tf.keras.models.load_model(
        best_result_file_name,
        custom_objects={'CopyChannels': cai.layers.CopyChannels}
    )

    # Make predictions
    y_pred_prob = model.predict(test_x)
    y_pred = np.argmax(y_pred_prob, axis=1)

    # Create the confusion matrix
    cm = confusion_matrix(np.argmax(test_y, axis=1), y_pred)

    # Visualize the confusion matrix using seaborn
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=False, fmt='d', cmap='Blues', cbar=False,
                xticklabels=label_of_classes, yticklabels=label_of_classes)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')

    figfilename = basefilename+'-CF.png'
    plt.savefig(figfilename, bbox_inches='tight')

    plt.show()